In [1]:
import numpy as np
import pandas as pd

import torch 
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

In [2]:
# Hyper Parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
train_df = pd.read_csv('./kaggle-mnist/train.csv')
test_df = pd.read_csv('./kaggle-mnist/test.csv')

In [4]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
test_df.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X_train = torch.Tensor(train_df.drop('label', axis=1).values.astype('float32').reshape(42000, 1, 28, 28))
y_train = torch.IntTensor(train_df['label'].values.astype('int32'))

train_dataset = TensorDataset(data_tensor=X_train, target_tensor=y_train)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

In [7]:
# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
        
cnn = CNN()

In [8]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

In [9]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Iter [100/420] Loss: 0.1287
Epoch [1/5], Iter [200/420] Loss: 0.1109
Epoch [1/5], Iter [300/420] Loss: 0.0413
Epoch [1/5], Iter [400/420] Loss: 0.0452
Epoch [2/5], Iter [100/420] Loss: 0.0210
Epoch [2/5], Iter [200/420] Loss: 0.0621
Epoch [2/5], Iter [300/420] Loss: 0.0236
Epoch [2/5], Iter [400/420] Loss: 0.0340
Epoch [3/5], Iter [100/420] Loss: 0.0252
Epoch [3/5], Iter [200/420] Loss: 0.0425
Epoch [3/5], Iter [300/420] Loss: 0.0672
Epoch [3/5], Iter [400/420] Loss: 0.0648
Epoch [4/5], Iter [100/420] Loss: 0.0460
Epoch [4/5], Iter [200/420] Loss: 0.0360
Epoch [4/5], Iter [300/420] Loss: 0.0362
Epoch [4/5], Iter [400/420] Loss: 0.0308
Epoch [5/5], Iter [100/420] Loss: 0.0049
Epoch [5/5], Iter [200/420] Loss: 0.0202
Epoch [5/5], Iter [300/420] Loss: 0.0207
Epoch [5/5], Iter [400/420] Loss: 0.0041


In [ ]:
# 学習したモデルを使ってテストデータ 'test.csv' を分類、Kaggleに提出までやってください